# Chat Memory

In [1]:
from haystack_experimental.chat_message_stores.in_memory import InMemoryChatMessageStore
from haystack_experimental.components.retrievers import ChatMessageRetriever
from haystack_experimental.components.writers import ChatMessageWriter
from haystack.dataclasses import ChatMessage
from haystack.components.joiners import ListJoiner
from haystack import Pipeline
from typing import List
from haystack.components.builders import ChatPromptBuilder, PromptBuilder
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.generators import OpenAIGenerator
from haystack.components.converters import OutputAdapter
from haystack.utils import Secret
from getpass import getpass
import os

In [2]:
os.environ["OPENAI_API_KEY"] = getpass("Masukkan OpenAI API Key Anda: ")

In [3]:
os.environ["MONGO_CONNECTION_STRING"] = getpass("Masukkan MongoDB Connection String Anda: ")

In [4]:
system_message = ChatMessage.from_system("You are a helpful assistant that answers questions based on the provided context.")
user_message_template = """
Answer the question based on the user query:
query:{{query}}
answer:
"""
user_message = ChatMessage.from_user(user_message_template)

In [5]:
pipeline = Pipeline()
pipeline.add_component("prompt_builder", ChatPromptBuilder(variables=["query"], required_variables=["query"]))
pipeline.add_component("generator", OpenAIChatGenerator(model="gpt-4.1", api_key=Secret.from_token(os.environ["OPENAI_API_KEY"])))


pipeline.connect("prompt_builder.prompt", "generator.messages")

🚅 Components
  - prompt_builder: ChatPromptBuilder
  - generator: OpenAIChatGenerator
🛤️ Connections
  - prompt_builder.prompt -> generator.messages (List[ChatMessage])

In [6]:
while True:
    messages = [system_message, user_message]
    query = input("Please input your question or type 'exit' to quit.\n")
    if query.lower() == "exit":
        break
    res = pipeline.run(
        {
            "prompt_builder": {
                "query": query,
                "template":messages
            }
        }
    )
    print("AI Response:", res["generator"]["replies"][0].text)

AI Response: AI, or Artificial Intelligence, refers to the ability of machines or computer systems to perform tasks that typically require human intelligence. These tasks can include learning, problem-solving, understanding language, recognizing patterns and images, making decisions, and more. AI systems use algorithms and models to analyze data, adapt to new information, and carry out complex functions automatically.


In [7]:
memory_store = InMemoryChatMessageStore()
memory_retriever = ChatMessageRetriever(memory_store)
memory_writer = ChatMessageWriter(memory_store)

In [8]:
system_message = ChatMessage.from_system("You are a helpful assistant that answers questions based on the provided context.")
user_message_template = """
Answer the question based on the user query, please pay attention to the chat history:
chat_history:
{% for memory in memories %}
    {{memory.text}}
{% endfor %}

query:{{query}}
answer:
"""
user_message = ChatMessage.from_user(user_message_template)

In [9]:
pipeline = Pipeline()
pipeline.add_component("prompt_builder", ChatPromptBuilder(variables=["query","memories"], required_variables=["query","memories"]))
pipeline.add_component("generator", OpenAIChatGenerator(model="gpt-4.1", api_key=Secret.from_token(os.environ["OPENAI_API_KEY"])))
pipeline.add_component("joiner", ListJoiner(List[ChatMessage]))
pipeline.add_component("memory_retriever", memory_retriever)
pipeline.add_component("memory_writer", memory_writer)

pipeline.connect("prompt_builder.prompt", "generator.messages")
pipeline.connect("generator.replies", "joiner")
pipeline.connect("joiner", "memory_writer")
pipeline.connect("memory_retriever", "prompt_builder.memories")

🚅 Components
  - prompt_builder: ChatPromptBuilder
  - generator: OpenAIChatGenerator
  - joiner: ListJoiner
  - memory_retriever: ChatMessageRetriever
  - memory_writer: ChatMessageWriter
🛤️ Connections
  - prompt_builder.prompt -> generator.messages (List[ChatMessage])
  - generator.replies -> joiner.values (List[ChatMessage])
  - joiner.values -> memory_writer.messages (List[ChatMessage])
  - memory_retriever.messages -> prompt_builder.memories (List[ChatMessage])

In [10]:
while True:
    messages = [system_message, user_message]
    query = input("Please input your question or type 'exit' to quit.\n")
    if query.lower() == "exit":
        break
    res = pipeline.run(
        data={
            "prompt_builder": {
                "query": query,
                "template":messages
            },
            "joiner":{
                "values": [ChatMessage.from_user(query)]
            }
        },
        include_outputs_from=["generator"]
    )
    # print(res)
    print("AI Response:", res["generator"]["replies"][0].text)

AI Response: The CEO of Tesla is Elon Musk.
AI Response: Elon Musk was born in Pretoria, South Africa.
